In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import json
import yaml

with open("chat_app.yaml") as f:
    spec1 = json.dumps(yaml.load(f, Loader=yaml.Loader))

with open("other_app.yaml") as f:
    spec2 = json.dumps(yaml.load(f, Loader=yaml.Loader))

In [ ]:
spec1

In [ ]:
from langchain_community.utilities.openapi import OpenAPISpec

parsed_spec1 = OpenAPISpec.from_text(spec1)
parsed_spec2 = OpenAPISpec.from_text(spec2)

In [ ]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn

openai_fns1, call_api_fn1 = openapi_spec_to_openai_fn(parsed_spec1)
tools1 = [
    {"type": "function", "function": fn}
    for fn in openai_fns1
]
openai_fns2, call_api_fn2  = openapi_spec_to_openai_fn(parsed_spec2)
tools2 = [
    {"type": "function", "function": fn}
    for fn in openai_fns2
]
final_tools = tools1 + tools2

In [ ]:
tools2

In [ ]:
from langchain_openai import ChatOpenAI

from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage

llm_with_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(final_tools)

In [ ]:
sys_msg = SystemMessage(content="Use the provided APIs to respond to this user query")

In [ ]:
hu_msg = HumanMessage(content="Gib mir alle Dokumente von John Doe")
messages = [sys_msg, hu_msg]

In [ ]:
print(messages)

In [ ]:
llm_output = llm_with_tools.invoke(input=messages)
messages.append(llm_output)

In [ ]:
llm_output.tool_calls

In [ ]:
tool_mapping = {
    tool["function"]["name"]: (
        lambda fn_args, call_fn=(call_api_fn1 if tool["function"]["name"].startswith("chat") else call_api_fn2), tool_name=tool["function"]["name"]: call_fn(tool_name, fn_args)
    )
    for tool in final_tools
}

In [ ]:
tool_mapping

In [ ]:
for tool_call in llm_output.tool_calls:
    selected_tool = tool_mapping[tool_call["name"].lower()]
    response = selected_tool(tool_call["args"])

    if hasattr(response, 'json'):
        tool_output = response.json()
    elif hasattr(response, 'text'):
        tool_output = response.text
    else:
        tool_output = response

    try:
        tool_output = json.dumps(tool_output, indent=2, ensure_ascii=False)
    except (TypeError, ValueError):
        tool_output = str(tool_output)

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))


In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages)